In [1]:
from language_detection.data import load_wili_2018_dataset

In [2]:
wili_2018_data_path = "/home/derek/PythonProjects/language_detection/datasets/WiLi_2018"

In [3]:
wiki_dataset = load_wili_2018_dataset(wili_2018_data_path)

2023-11-26 12:19:50.193 | INFO     | language_detection.data.loaders:load_wili_2018_dataset:67 - 'drop_duplicates' is true, dropping duplicates from *training* set...
2023-11-26 12:19:50.257 | INFO     | language_detection.data.loaders:load_wili_2018_dataset:74 - dropped 3117 samples from training data that also appeared in the test data


In [4]:
len(set(wiki_dataset.x_train).intersection(set(wiki_dataset.x_test)))

0

In [5]:
wiki_dataset.__annotations__

{'x_train': list[str],
 'x_test': list[str],
 'y_train': list[str],
 'y_test': list[str],
 'idx2lang': dict[int, str],
 'lang2idx': dict[str, int],
 'labels': dict[str, str] | None,
 'dropped': list[str] | None}

In [6]:
wiki_dataset.x_train[:3]

['Klement Gottwaldi surnukeha palsameeriti ning paigutati mausoleumi. Surnukeha oli aga liiga hilja ja oskamatult palsameeritud ning hakkas ilmutama lagunemise tundemärke. 1962. aastal viidi ta surnukeha mausoleumist ära ja kremeeriti. Zlíni linn kandis aastatel 1949–1989 nime Gottwaldov. Ukrainas Harkivi oblastis kandis Zmiivi linn aastatel 1976–1990 nime Gotvald.',
 'Sebes, Joseph; Pereira Thomas (1961) (på eng). The Jesuits and the Sino-Russian treaty of Nerchinsk (1689): the diary of Thomas Pereira. Bibliotheca Instituti historici S. I., 99-0105377-3 ; 18. Rome. Libris 677492',
 'भारतीय स्वातन्त्र्य आन्दोलन राष्ट्रीय एवम क्षेत्रीय आह्वान, उत्तेजनासभ एवम प्रयत्नसँ प्रेरित, भारतीय राजनैतिक सङ्गठनद्वारा सञ्चालित अहिंसावादी आ सैन्यवादी आन्दोलन छल, जेकर एक समान उद्देश्य, अङ्ग्रेजी शासनक भारतीय उपमहाद्वीपसँ जडीसँ उखाड फेकनाई छल। ई आन्दोलनक शुरुआत १८५७ मे भेल सिपाही विद्रोहक मानल जाइत अछि। स्वाधीनताक लेल हजारो लोग अपन प्राणक बलि देलक। भारतीय राष्ट्रीय कांग्रेस १९३० कांग्रेस अधिवेशन मे अङ्

In [8]:
import torch

In [9]:
a = torch.LongTensor([1])

In [10]:
a

tensor([1])

In [11]:
a.tolist()

[1]

In [12]:
memoryview(bytes("hello, world", encoding="utf8")).tolist()

[104, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100]

In [13]:
import numpy as np
from numpy.typing import ArrayLike

In [120]:
text = wiki_dataset.x_train[2]
mask_pct = 0.15

# convert to integer sequence of bytes
byte_sequence: np.ndarray = np.array(memoryview(bytes(text, encoding="utf8")).tolist())
# offset by 4 (for pad=0, cls=1, eos=2, and mask=3)
byte_sequence += 4
# add cls (1) and eos (2)
target_sequence = np.concatenate((np.array([1]), byte_sequence, np.array([2])))
input_sequence = np.copy(target_sequence)
# BERT mask inputs at mask_pct rate. of those, mask 80%, random value 10%, keep 10%
mod_count = int(np.round(len(byte_sequence)*mask_pct))
to_mod_idxs = np.random.choice(np.arange(1, len(byte_sequence)-2), replace=False, size=mod_count)
split_indices = np.cumsum([int(len(to_mod_idxs)*0.80), int(len(to_mod_idxs)*0.10), int(len(to_mod_idxs)*0.10)])
split_indices[-1] = len(to_mod_idxs)
mask_idxs, repl_idxs, keep_idxs, _ = np.split(to_mod_idxs, split_indices)
target_sequence[mask_idxs] = 2
target_sequence[repl_idxs] = np.random.randint(4, 256+4, size=(len(repl_idxs),))
modded_idxs = np.sort(to_mod_idxs)

# check
diff_idx = np.where(np.invert(np.isclose(input_sequence, target_sequence)))
okay = np.isin(diff_idx, modded_idxs).all(1)[0]
assert okay


True


In [15]:
# text transformation function
def transform_text(text: str):
    byte_sequence: ArrayLike = np.array(memoryview(bytes(text, encoding="utf8")).tolist())
